<a href="https://colab.research.google.com/github/Sundaynot/Big_data_project/blob/main/Big_data_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>